In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lactose_Intolerance"
cohort = "GSE136395"

# Input paths
in_trait_dir = "../../input/GEO/Lactose_Intolerance"
in_cohort_dir = "../../input/GEO/Lactose_Intolerance/GSE136395"

# Output paths
out_data_file = "../../output/preprocess/Lactose_Intolerance/GSE136395.csv"
out_gene_data_file = "../../output/preprocess/Lactose_Intolerance/gene_data/GSE136395.csv"
out_clinical_data_file = "../../output/preprocess/Lactose_Intolerance/clinical_data/GSE136395.csv"
json_path = "../../output/preprocess/Lactose_Intolerance/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effects of a novel oral nutritional supplement as compared to standard care on body composition, physical function and skeletal muscle mRNA expression in Dutch older adults with (or at risk of) undernutrition"
!Series_summary	"In a randomized controlled trial, 82 older adults (>65y) with (or at risk of) undernutrition (n=82) were randomly allocated to 12 weeks of supplementation with a novel supplement (586 kcal, 22 g protein of which 50% whey and 50% casein, 206 mg ursolic acid, 7 g free BCAAs, 11 µg vitamin D) or standard care (600 kcal, 24g protein of which 100% casein, 4 µg vitamin D). Body weight increased significantly in the 12 weeks, both in the intervention group (+1.6 ± 0.2 kg, p<.0001) and in the standard care group (+1.8 ± 0.2 kg, p<.0001). Gait speed during 4m and 400m tests improved over time in the intervention group, whereas the standard care showed no improvements (time*treatment effects 400m: p=0.038 and 4m: p=0.048). Gene se

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this study involves "Microarray analysis" of skeletal muscle biopsies
# This indicates gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait - we'll use the "experimental condition" as our trait row
# Row 5 shows two values: 'novel oral nutritional supplement' and 'standard-care nutritional supplement'
trait_row = 5

# For age - present in row 2
age_row = 2

# For gender - present in row 0
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert experimental condition to binary: 1 for novel supplement, 0 for standard care"""
    if not isinstance(value, str):
        return None
    
    value_lower = value.lower()
    if 'novel' in value_lower:
        return 1
    elif 'standard' in value_lower:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to continuous value"""
    if not isinstance(value, str):
        return None
    
    # Extract the numeric part after the colon
    try:
        # Handle format like "age (yrs): 70"
        value_parts = value.split(': ')
        if len(value_parts) > 1:
            age_str = value_parts[1].strip()
            age = float(age_str)
            # There appears to be an error in the data where one age is listed as 5
            # This is likely a typo since all other ages are 65+
            if age < 20:  # Assume any age under 20 is an error for older adults
                return None
            return age
        return None
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if not isinstance(value, str):
        return None
    
    # The format appears to be "sex (female=1, male=0): 1" or "sex (female=1, male=0): 0"
    # Note: The original data has female=1, male=0, but we need to convert to female=0, male=1
    try:
        value_parts = value.split(': ')
        if len(value_parts) > 1:
            gender_value = value_parts[1].strip()
            # Since in the data female=1 and male=0, we need to flip these values
            if gender_value == '1':  # Female in original data
                return 0
            elif gender_value == '0':  # Male in original data
                return 1
        return None
    except (ValueError, IndexError):
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Define clinical_data (assuming it was created in a previous step)
    clinical_data = pd.DataFrame.from_dict(
        {i: values for i, values in 
         {0: ['sex (female=1, male=0): 1', 'sex (female=1, male=0): 0'], 
          1: ['subjectid: 202', 'subjectid: 203', 'subjectid: 205', 'subjectid: 211', 'subjectid: 212', 'subjectid: 214', 'subjectid: 215', 'subjectid: 219', 'subjectid: 231', 'subjectid: 238', 'subjectid: 243', 'subjectid: 245', 'subjectid: 250', 'subjectid: 252', 'subjectid: 253', 'subjectid: 258', 'subjectid: 259', 'subjectid: 261', 'subjectid: 264', 'subjectid: 265', 'subjectid: 266'], 
          2: ['age (yrs): 70', 'age (yrs): 66', 'age (yrs): 74', 'age (yrs): 69', 'age (yrs): 83', 'age (yrs): 5', 'age (yrs): 77', 'age (yrs): 75', 'age (yrs): 72', 'age (yrs): 71', 'age (yrs): 68', 'age (yrs): 80'], 
          3: ['moment of sampling (pre/post intervention): pre-intervention (at baseline)', 'moment of sampling (pre/post intervention): post-intervention (after 12 wks)'], 
          4: ['time of sampling (hr: min): 11:35', 'time of sampling (hr: min): 10:40', 'time of sampling (hr: min): 10:55', 'time of sampling (hr: min): 10:25', 'time of sampling (hr: min): 10:30', 'time of sampling (hr: min): 11:45', 'time of sampling (hr: min): 10:20', 'time of sampling (hr: min): 11:40', 'time of sampling (hr: min): 10:44', 'time of sampling (hr: min): 11:15', 'time of sampling (hr: min): 11:20', 'time of sampling (hr: min): 12:40', 'time of sampling (hr: min): 11:25', 'time of sampling (hr: min): 12:20', 'time of sampling (hr: min): 11:30', 'time of sampling (hr: min): 11:06', 'time of sampling (hr: min): 11:23', 'time of sampling (hr: min): 11:10', 'time of sampling (hr: min): 12:25', 'time of sampling (hr: min): 10:35', 'time of sampling (hr: min): 11:50', 'time of sampling (hr: min): 11:00', 'time of sampling (hr: min): 12:50', 'time of sampling (hr: min): 10:05', 'time of sampling (hr: min): 12:03'], 
          5: ['experimental condition: novel oral nutritional supplement', 'experimental condition: standard-care nutritional supplement'], 
          6: ['tissue: skeletal muscle'], 
          7: ['sample type: non-fasted morning sample']}.items()
    }, orient='index')
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical Features Preview:
{0: [1.0, 70.0, 0.0], 1: [0.0, 66.0, 1.0], 2: [nan, 74.0, nan], 3: [nan, 69.0, nan], 4: [nan, 83.0, nan], 5: [nan, nan, nan], 6: [nan, 77.0, nan], 7: [nan, 75.0, nan], 8: [nan, 72.0, nan], 9: [nan, 71.0, nan], 10: [nan, 68.0, nan], 11: [nan, 80.0, nan], 12: [nan, nan, nan], 13: [nan, nan, nan], 14: [nan, nan, nan], 15: [nan, nan, nan], 16: [nan, nan, nan], 17: [nan, nan, nan], 18: [nan, nan, nan], 19: [nan, nan, nan], 20: [nan, nan, nan], 21: [nan, nan, nan], 22: [nan, nan, nan], 23: [nan, nan, nan], 24: [nan, nan, nan]}
Clinical features saved to ../../output/preprocess/Lactose_Intolerance/clinical_data/GSE136395.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"The effects of a novel oral nutritional supplement as compared to standard care on body composition, physical function and skeletal muscle mRNA expression in Dutch older adults with (or at risk of) undernutrition"
Line 1: !Series_geo_accession	"GSE136395"
Line 2: !Series_status	"Public on Apr 06 2021"
Line 3: !Series_submission_date	"Aug 27 2019"
Line 4: !Series_last_update_date	"Apr 09 2021"
Line 5: !Series_pubmed_id	"33799307"
Line 6: !Series_summary	"In a randomized controlled trial, 82 older adults (>65y) with (or at risk of) undernutrition (n=82) were randomly allocated to 12 weeks of supplementation with a novel supplement (586 kcal, 22 g protein of which 50% whey and 50% casein, 206 mg ursolic acid, 7 g free BCAAs, 11 µg vitamin D) or standard care (600 kcal, 24g protein of which 100% casein, 4 µg vitamin D). Body weight increased significantly in the 12 weeks, both in the intervention group (+1.6 ± 0.2 kg, p<.0001) and i

Successfully extracted gene data with 53617 rows
First 20 gene IDs:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on examining the gene identifiers such as '16650001', '16650003', etc., these appear to be 
# probe identifiers from an Illumina microarray platform rather than human gene symbols.
#
# These numeric identifiers (starting with 1665...) are typical of Illumina BeadArray probes
# and will need to be mapped to standard human gene symbols for interpretability and
# cross-study compatibility.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 2305573 rows

Gene annotation preview (first few rows):
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box h

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# From examining the gene_annotation and gene_data, we need to map:
#   - The 'ID' column in gene_annotation (which contains probes like '16650001')
#   - To gene symbols found in the 'gene_assignment' column

# Extract IDs and gene_assignment for mapping
print("Creating gene mapping from probes to gene symbols...")
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'gene_assignment'  # Column containing gene symbols/names

# 2. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Generated mapping with {len(gene_mapping)} rows")

# Preview the mapping to verify
print("\nPreview of gene mapping:")
print(preview_df(gene_mapping))

# Verify the gene data before mapping
print("\nGene data shape before mapping:", gene_data.shape)

# 3. Convert probe-level measurements to gene expression data
print("\nApplying gene mapping to convert probe IDs to gene symbols...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Successfully mapped gene data with {len(gene_data.index)} genes")

# Show example of gene symbols in the resulting dataframe
print("\nFirst 20 gene symbols in mapped gene expression data:")
print(gene_data.index[:20])

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Creating gene mapping from probes to gene symbols...
Generated mapping with 53617 rows

Preview of gene mapping:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_051986 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 

Successfully mapped gene data with 81076 genes

First 20 gene symbols in mapped gene expression data:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1',
       'A1-', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18'],
      dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Lactose_Intolerance/gene_data/GSE136395.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Save the normalized gene expression data from the previous step
print("\nSaving normalized gene data...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical data from the matrix file
print("\nExtracting clinical data...")
try:
    # Get the file paths again to make sure we have them
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract background information and clinical data
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # Process clinical data using trait information from Step 2
    trait_row = 1  # Based on analysis in step 2 - group (OW/OB vs NW/MONW)
    gender_row = 0  # Gender data
    age_row = 2     # Age data
    
    # Define conversion functions based on Step 2
    def convert_trait(value):
        """Convert trait value (binary: 1 for OW/OB, 0 for NW/MONW)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'OW/OB' in value:
            return 1  # Overweight/Obese is associated with higher LDL cholesterol
        elif 'NW' in value or 'MONW' in value:
            return 0  # Normal weight (includes metabolically obese normal weight)
        else:
            return None

    def convert_gender(value):
        """Convert gender value to binary (0: female, 1: male)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert gender
        if value.lower() == 'woman':
            return 0
        elif value.lower() == 'man':
            return 1
        else:
            return None
    
    def convert_age(value):
        """Convert age value to float"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)  # Convert to float for continuous variable
        except:
            return None
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Check if we have valid trait information
    is_trait_available = trait_row is not None and not selected_clinical_df.loc[trait].isnull().all()
    print(f"Trait information available: {is_trait_available}")
    
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Saving normalized gene data...


Normalized gene data saved to ../../output/preprocess/Lactose_Intolerance/gene_data/GSE136395.csv

Extracting clinical data...
Clinical data saved to ../../output/preprocess/Lactose_Intolerance/clinical_data/GSE136395.csv
Clinical data shape: (3, 42)
Trait information available: False

Linking clinical and genetic data...
Cannot link data: clinical data is not available

Skipping missing value handling and bias evaluation as linked data is not available

Performing final validation...
Abnormality detected in the cohort: GSE136395. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Lactose_Intolerance/cohort_info.json

Dataset usability for Lactose_Intolerance association studies: False
Reason: Dataset does not contain required trait information
